# Double Pendulum System
## Computational Physics Final Project
#### by Irene Yang (ay2431@nyu.edu) and Lea Friedrich (flf7670@nyu.edu)

Solving the Lagrangian for the double pendulum, we can get the following four first-order differential equation.

$ \dot{\theta}_1(t)=\omega_1(t) $

$ \dot{\theta}_2(t)=\omega_2(t) $

$ \dot{\omega}_1(t)= f_1(\theta_1, \theta_2, \omega_1, \omega_2) = \frac{m_2 l_1 \omega_1^2 \sin(\Delta \theta)\cos(\Delta\theta) + m_2l_2\omega_2^2\sin(\Delta\theta) + m_2g\sin(\theta_2)\cos(\Delta\theta) - Mg\sin(\theta_1)}{l_1(m_1+m_2\sin^2(\Delta\theta))} $

$ \dot{\omega}_2(t)= f_2(\theta_1, \theta_2, \omega_1, \omega_2) = \frac{-M l_1 \omega_1^2 \sin(\Delta \theta) - m_2l_2\omega_2^2\sin(\Delta\theta)\cos(\Delta\theta) + Mg\sin(\theta_1)\cos(\Delta\theta) - Mg\sin(\theta_2)}{l_2(m_1+m_2\sin^2(\Delta\theta))} $

Where, $M=m_1+m_2$ and $\Delta\theta = \theta_2 - \theta_1$

In [1]:
# Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc
import matplotlib as mpl

In [2]:
# Get the time derivative functions of each angular velocity from given parameters
def get_f1(m1: float, m2: float, l1: float, l2: float):
  M = m1 + m2
  g = 9.81
  return lambda u: ( m2*l1*(u[2]**2)*np.sin(u[1]-u[0])*np.cos(u[1]-u[0]) + m2*l2*(u[3]**2)*np.sin(u[1]-u[0]) + m2*g*np.sin(u[1])*np.cos(u[1]-u[0]) - M*g*np.sin(u[0]) ) / (l1*(m1+m2*(np.sin(u[1]-u[0])**2)))

def get_f2(m1: float, m2: float, l1: float, l2: float):
  M = m1 + m2
  g = 9.81
  return lambda u: ( -M*l1*(u[2]**2)*np.sin(u[1]-u[0]) - m2*l2*(u[3]**2)*np.sin(u[1]-u[0])*np.cos(u[1]-u[0]) + M*g*np.sin(u[0])*np.cos(u[1]-u[0]) - M*g*np.sin(u[1]) ) / (l2*(m1+m2*(np.sin(u[1]-u[0])**2)))

In [3]:
#------------------------------------------------------------
# Using RK4 method to estimate vector value at next timestep
#------------------------------------------------------------
def next_timestep(u0: np.array, f1: callable, f2: callable, h: float):
  # Function Variables
    # u0 - list(vector) of initial theta and omega values [theta_1, theta_2, omega_1, omega_2]
    # f1 - time derivative function of omega_1, takes array as argument
    # f2 - time derivative function of omega_2, takes array as argument
    # h  - timestep length

  k1 = np.array([u0[2], u0[3], f1(u0), f2(u0)])
  u1 = u0 + h/2*k1

  k2 = np.array([u1[2], u1[3], f1(u1), f2(u1)])
  u2 = u0 + h/2*k2

  k3 = np.array([u2[2], u2[3], f1(u2), f2(u2)])
  u3 = u0 + h*k3

  k4 = np.array([u3[2], u3[3], f1(u3), f2(u3)])

  k = (k1 + 2*k2 + 2*k3 + k4)/6
  u = u0 + h*k

  return u

In [ ]:
#-------------------------------------------------------------------------------------------------------
# Retrieving estimated vector (theta_1, theta_2, omega_1, omega_2) values at each time point specified
#-------------------------------------------------------------------------------------------------------

def evolve(T: float, N: int, param: list, u0: np.array):
  # Function Variables
    # T - Total duration of time over which system is evolved
    # N - Total number of timesteps to evaluate
    # param - parameters of the system [ m1, m2, l1, l2 ]
    # u0 - initial values [ theta_1, theta_2, omega_1, omega_2 ]

  m1, m2, l1, l2 = param

  f1 = get_f1(m1,m2,l1,l2)
  f2 = get_f2(m1,m2,l1,l2)
  h = T/N

  t_ls = np.arange(0,T+h/2,h)
  u_ls = [u0]

  for i in range(N):
    next_u = next_timestep(u_ls[-1], f1, f2, h)
    u_ls.append(next_u)

  u_ls = np.array(u_ls)
  u0_ls = u_ls[:,0]
  u1_ls = u_ls[:,1]
  u2_ls = u_ls[:,2]
  u3_ls = u_ls[:,3]

  return t_ls, u0_ls, u1_ls, u2_ls, u3_ls

In [ ]:
def plot_energy(t_ls: np.array, theta1_ls: np.array, theta2_ls: np.array, omega1_ls: np.array, omega2_ls: np.array, initial: np.array, g=9.81):
  # position of mass 1
  x1_ls = l1*np.sin(theta1_ls)
  y1_ls = -l1*np.cos(theta1_ls)
  # position of mass 2
  x2_ls = x1_ls + l2*np.sin(theta2_ls)
  y2_ls = y1_ls -l2*np.cos(theta2_ls)

  # Plot energy of pendulum ------------------------------------------
  # ideally, this should be constant throughout the evolution

  # Potential energy of the system
  U_ls = m1*g*y1_ls + m2*g*y2_ls

  # velocities of mass 1
  vx1 = l1*np.cos(theta1_ls) * omega1_ls
  vy1 = l1*np.sin(theta1_ls) * omega1_ls
  # velocities of mass 2
  vx2 = vx1 + l2*np.cos(theta2_ls) * omega2_ls
  vy2 = vy1 + l2*np.sin(theta2_ls) * omega2_ls

  # Kinetic energy of the system
  K_ls = 0.5*m1*(vx1**2 + vy1**2) + 0.5*m2*(vx2**2 + vy2**2)
  E_ls = U_ls + K_ls

  initial_E = E_ls[0]
  expected_E_ls = np.full_like(E_ls, initial_E) #np.array([initial_E]*len(t_ls))

  deviation = np.abs(E_ls - expected_E_ls)
  max_deviation = np.max(deviation)


  print(f"Parameters: m1 = {m1}kg, m2 = {m2}kg, l1 = {l1}m, l2 = {l2}m")
  print(f"Initial condition:  theta_1 = {initial[0]:.3f}, theta_2 = {initial[1]:.3f}, omega_1 = {initial[2]:.3f}, omega_2 = {initial[3]:.3f}")
  print(f"Time step size h = {T/N}")
  print(f"Max deviation ∆E = {max_deviation:.3e}J ({max_deviation/initial_E*100:.3}%)")

  # making plots
  fig, ax = plt.subplots()

  plt.plot(t_ls, E_ls, label='Energy of simulated system')
  plt.plot(t_ls, expected_E_ls, linestyle='--', label='Expected Energy')
  plt.legend(loc='lower left')
  plt.xlabel("Time (s)")
  plt.ylabel("Energy (J)")
  # plt.ylim(min(initial_E*1.005, initial_E*0.995), max(initial_E*1.005, initial_E*0.995))
  plt.title("Energy of Double pendulum over time")
  plt.show()


In [ ]:
#------------------------------------------------------------
# Animate Double Pendulum
#------------------------------------------------------------

def animate(theta1_ls: np.array, theta2_ls: np.array, print=False):
  # position of mass 1
  x1_ls = l1*np.sin(theta1_ls)
  y1_ls = -l1*np.cos(theta1_ls)
  # position of mass 2
  x2_ls = x1_ls + l2*np.sin(theta2_ls)
  y2_ls = y1_ls -l2*np.cos(theta2_ls)

  # Animate motion of pendulum ------------------------------------------
  mpl.rcParams['animation.embed_limit'] = 200  # MB - To ensure we get long enough animation
  nsteps = len(x1_ls)

  fig, ax = plt.subplots()
  ax.set_aspect('equal')

  R = 1.3*(l1+l2)
  ax.set_xlim(-R, R)
  ax.set_ylim(-R, R)

  #three points, pivot mass1 and mass2
  line, = ax.plot([],[],'o-', linewidth=3.0)

  #trail of mass 2
  trace, = ax.plot([],[],'.-', markersize=1, alpha=0.5, color='blue')
  trace_x, trace_y = [],[]

  if print:
    print(f"Parameters: m1 = {m1}kg, m2 = {m2}kg, l1 = {l1}m, l2 = {l2}m")
    print(f"Initial condition:  theta_1 = {initial[0]:.3f}, theta_2 = {initial[1]:.3f}, omega_1 = {initial[2]:.3f}, omega_2 = {initial[3]:.3f}")
    print()
  plt.title("Double pendulum animation")

  def update(i):
      x_now=[0.0, x1_ls[i], x2_ls[i]]
      y_now=[0.0, y1_ls[i], y2_ls[i]]
      line.set_data(x_now, y_now)

      trace_x.append(x2_ls[i])
      trace_y.append(y2_ls[i])
      trace.set_data(trace_x, trace_y)

      return line, trace

  rc('animation', html='jshtml')

  frame_len = T/N*10**3 #ms
  ani = FuncAnimation(fig, update, frames=nsteps, interval=frame_len, blit=True)
  return ani

In [ ]:
def get_MaxEdev(theta1_ls: np.array, theta2_ls: np.array, omega1_ls: np.array, omega2_ls: np.array, param: list, g=9.81):
  m1, m2, l1, l2 = param

  # position of mass 1
  x1_ls = l1*np.sin(theta1_ls)
  y1_ls = -l1*np.cos(theta1_ls)
  # position of mass 2
  x2_ls = x1_ls + l2*np.sin(theta2_ls)
  y2_ls = y1_ls -l2*np.cos(theta2_ls)

  # Solve for energy of pendulum ------------------------------------------
  # Potential energy of the system
  U_ls = m1*g*y1_ls + m2*g*y2_ls

  # velocities of mass 1
  vx1 = l1*np.cos(theta1_ls) * omega1_ls
  vy1 = l1*np.sin(theta1_ls) * omega1_ls
  # velocities of mass 2
  vx2 = vx1 + l2*np.cos(theta2_ls) * omega2_ls
  vy2 = vy1 + l2*np.sin(theta2_ls) * omega2_ls

  # Kinetic energy of the system
  K_ls = 0.5*m1*(vx1**2 + vy1**2) + 0.5*m2*(vx2**2 + vy2**2)
  E_ls = U_ls + K_ls

  initial_E = E_ls[0]
  expected_E_ls = np.full_like(E_ls, initial_E) #np.array([initial_E]*len(t_ls))

  deviation = np.abs(E_ls - expected_E_ls)
  max_dev = np.max(deviation)
  dev_percent = max_dev / np.abs(initial_E) * 100

  return max_dev, dev_percent

In [ ]:
#------------------------------------------------------------
# Simulation 1 - Double Pendulum animation
#------------------------------------------------------------

# Parameters and initial data ----------------------------------------------
g = 9.81 #m/s^2

m1 = 2 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 1 # m

T = 30 #s
N = 1500

initial1 = np.array([np.pi/2, np.pi, 0, -2])

# Solve the pendulum motion and plot energy ---------------------------------

param1 = [m1, m2, l1, l2]
t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls = evolve(T, N, param1, initial1)

plot_energy(t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls, initial1)

animation1 = animate(theta1_ls, theta2_ls)

animation1.save(f"Simulation1_DoublePendulum_param={param1}_initial={initial1}.mp4", fps=40, writer="ffmpeg")
animation1

In [ ]:
#------------------------------------------------------------
# Simulation 2 - Double Pendulum animation
#------------------------------------------------------------

# Parameters and initial data ----------------------------------------------
g = 9.81 #m/s^2

m1 = 1 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 2 # m

T = 30 #s
N = 1500

initial2 = np.array([np.pi, np.pi/2, 1, 0])

# Solve the pendulum motion and plot energy ---------------------------------

param2 = [m1, m2, l1, l2]
t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls = evolve(T, N, param2, initial2)

plot_energy(t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls, initial2)

animation2 = animate(theta1_ls, theta2_ls)

animation2.save(f"Simulation2_DoublePendulum_param={param2}_initial={initial2}.mp4", fps=40, writer="ffmpeg")
animation2

In [ ]:
#------------------------------------------------------------
# Simulation 3 - Compare trajectories of two simulations of close initial data
#------------------------------------------------------------

# Parameters and initial data ----------------------------------------------
g = 9.81 #m/s^2

m1 = 1 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 2 # m

T = 30 #s
N = 1500

initial3_1 = np.array([2/3*np.pi, np.pi/2, 1, 1])
initial3_2 = np.array([2/3*np.pi, np.pi/2, 1, 1.05])

# Solve the pendulum motion ---------------------------------
param3 = [m1, m2, l1, l2]

t_ls, theta1_ls_1, theta2_ls_1, omega1_ls_1, omega2_ls_1 = evolve(T, N, param3, initial3_1)
__ls, theta1_ls_2, theta2_ls_2, omega1_ls_2, omega2_ls_2 = evolve(T, N, param3, initial3_2)

# plot_energy(t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls, initial2)

# system 1
# position of mass 1
x1_ls_1 = l1*np.sin(theta1_ls_1)
y1_ls_1 = -l1*np.cos(theta1_ls_1)
# position of mass 2
x2_ls_1 = x1_ls_1 + l2*np.sin(theta2_ls_1)
y2_ls_1 = y1_ls_1 -l2*np.cos(theta2_ls_1)

# system 2
# position of mass 1
x1_ls_2 = l1*np.sin(theta1_ls_2)
y1_ls_2 = -l1*np.cos(theta1_ls_2)
# position of mass 2
x2_ls_2 = x1_ls_2 + l2*np.sin(theta2_ls_2)
y2_ls_2 = y1_ls_2 -l2*np.cos(theta2_ls_2)


# Animate motion of pendulum ------------------------------------------
mpl.rcParams['animation.embed_limit'] = 200  # MB - To ensure we get long enough animation
nsteps = len(x1_ls_1)

fig, ax = plt.subplots()
ax.set_aspect('equal')

R = 1.3*(l1+l2)
ax.set_xlim(-R, R)
ax.set_ylim(-R, R)

#three points, pivot mass1 and mass2
line1, = ax.plot([],[],'o-', linewidth=3.0) # system 1
line2, = ax.plot([],[],'o-', linewidth=3.0, color='red') # system 2

#trail of mass 2
trace1, = ax.plot([],[],'.-', markersize=1, alpha=0.5, color='blue') # system 1
trace_x1, trace_y1 = [],[]
trace2, = ax.plot([],[],'.-', markersize=1, alpha=0.5, color='magenta') # system 2
trace_x2, trace_y2 = [],[]

# if print:
#   print(f"Parameters: m1 = {m1}kg, m2 = {m2}kg, l1 = {l1}m, l2 = {l2}m")
#   print(f"Initial condition:  theta_1 = {initial[0]:.3f}, theta_2 = {initial[1]:.3f}, omega_1 = {initial[2]:.3f}, omega_2 = {initial[3]:.3f}")
#   print()
# plt.title("Double pendulum animation")

def update(i):
  #system 1
    x1_now=[0.0, x1_ls_1[i], x2_ls_1[i]]
    y1_now=[0.0, y1_ls_1[i], y2_ls_1[i]]
    line1.set_data(x1_now, y1_now)

    trace_x1.append(x2_ls_1[i])
    trace_y1.append(y2_ls_1[i])
    trace1.set_data(trace_x1, trace_y1)

  #system 2
    x2_now=[0.0, x1_ls_2[i], x2_ls_2[i]]
    y2_now=[0.0, y1_ls_2[i], y2_ls_2[i]]
    line2.set_data(x2_now, y2_now)

    trace_x2.append(x2_ls_2[i])
    trace_y2.append(y2_ls_2[i])
    trace2.set_data(trace_x2, trace_y2)

    return line1, line2, trace1, trace2

rc('animation', html='jshtml')

frame_len = T/N*10**3 #ms
animation3 = FuncAnimation(fig, update, frames=nsteps, interval=frame_len, blit=True)

animation3.save(f"Simulation3_compare_similar_initcond.mp4", fps=40, writer="ffmpeg")
animation3

In [ ]:
#------------------------------------------------------------
# Simulation 4 - Compare trajectories of two simulations of close initial data
#------------------------------------------------------------

# Parameters and initial data ----------------------------------------------
g = 9.81 #m/s^2

m1 = 1 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 2 # m

T = 30 #s
N = 1500

initial4_1 = np.array([2/3*np.pi, np.pi/2, 1, 0])
initial4_2 = np.array([2/3*np.pi, 1.05*np.pi/2, 1, 0])

# Solve the pendulum motion ---------------------------------
param4 = [m1, m2, l1, l2]

_, theta1_ls_1, theta2_ls_1, _, _ = evolve(T, N, param3, initial4_1)
_, theta1_ls_2, theta2_ls_2, _, _ = evolve(T, N, param3, initial4_2)

# plot_energy(t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls, initial2)

# system 1
# position of mass 1
x1_ls_1 = l1*np.sin(theta1_ls_1)
y1_ls_1 = -l1*np.cos(theta1_ls_1)
# position of mass 2
x2_ls_1 = x1_ls_1 + l2*np.sin(theta2_ls_1)
y2_ls_1 = y1_ls_1 -l2*np.cos(theta2_ls_1)

# system 2
# position of mass 1
x1_ls_2 = l1*np.sin(theta1_ls_2)
y1_ls_2 = -l1*np.cos(theta1_ls_2)
# position of mass 2
x2_ls_2 = x1_ls_2 + l2*np.sin(theta2_ls_2)
y2_ls_2 = y1_ls_2 -l2*np.cos(theta2_ls_2)


# Animate motion of pendulum ------------------------------------------
mpl.rcParams['animation.embed_limit'] = 200  # MB - To ensure we get long enough animation
nsteps = len(x1_ls_1)

fig, ax = plt.subplots()
ax.set_aspect('equal')

R = 1.3*(l1+l2)
ax.set_xlim(-R, R)
ax.set_ylim(-R, R)

#three points, pivot mass1 and mass2
line1, = ax.plot([],[],'o-', linewidth=3.0) # system 1
line2, = ax.plot([],[],'o-', linewidth=3.0, color='red') # system 2

#trail of mass 2
trace1, = ax.plot([],[],'.-', markersize=1, alpha=0.5, color='blue') # system 1
trace_x1, trace_y1 = [],[]
trace2, = ax.plot([],[],'.-', markersize=1, alpha=0.5, color='magenta') # system 2
trace_x2, trace_y2 = [],[]

# if print:
#   print(f"Parameters: m1 = {m1}kg, m2 = {m2}kg, l1 = {l1}m, l2 = {l2}m")
#   print(f"Initial condition:  theta_1 = {initial[0]:.3f}, theta_2 = {initial[1]:.3f}, omega_1 = {initial[2]:.3f}, omega_2 = {initial[3]:.3f}")
#   print()
# plt.title("Double pendulum animation")

def update(i):
  #system 1
    x1_now=[0.0, x1_ls_1[i], x2_ls_1[i]]
    y1_now=[0.0, y1_ls_1[i], y2_ls_1[i]]
    line1.set_data(x1_now, y1_now)

    trace_x1.append(x2_ls_1[i])
    trace_y1.append(y2_ls_1[i])
    trace1.set_data(trace_x1, trace_y1)

  #system 2
    x2_now=[0.0, x1_ls_2[i], x2_ls_2[i]]
    y2_now=[0.0, y1_ls_2[i], y2_ls_2[i]]
    line2.set_data(x2_now, y2_now)

    trace_x2.append(x2_ls_2[i])
    trace_y2.append(y2_ls_2[i])
    trace2.set_data(trace_x2, trace_y2)

    return line1, line2, trace1, trace2

rc('animation', html='jshtml')

frame_len = T/N*10**3 #ms
animation4 = FuncAnimation(fig, update, frames=nsteps, interval=frame_len, blit=True)

animation4.save(f"Simulation4_compare_similar_initcond.mp4", fps=40, writer="ffmpeg")
animation4

In [ ]:
# --------------------------------------------------------------------------------------------
# Plotting Maximum Energy deviation (error) for different step size
# --------------------------------------------------------------------------------------------

# Parameters
g = 9.81 #m/s^2

m1 = 1 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 2 # m

# Initial data
init_th1 = np.pi/2
init_th2 = np.pi
init_om1 = -1
init_om2 = 1

T = 10 #s
N_ls = np.array([50, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000])
h_ls = T/N_ls
Edev_array = np.zeros((len(h_ls)), dtype=float)

for i in range(len(N_ls)):
    _, theta1_ls, theta2_ls, omega1_ls, omega2_ls = evolve(T, N_ls[i], [m1, m2, l1, l2], [init_th1, init_th2, init_om1, init_om2])
    dev, percent = get_MaxEdev(theta1_ls, theta2_ls, omega1_ls, omega2_ls, [m1, m2, l1, l2])
    Edev_array[i] = percent

# print(h_ls)
# print(Edev_array)

m, b = np.polyfit(np.log(h_ls[:8]), np.log(Edev_array[:8]), 1)
fig, ax = plt.subplots(figsize=(7, 5))
plt.plot(h_ls, Edev_array, marker='.', linestyle='', label='sample points')
plt.plot(h_ls[:8], (np.e**b)*(h_ls[:8]**m), marker='', color = 'orange', label='line of best fit (approximation error)')
plt.legend(loc='lower right')

const = np.e**b
print(f"Relationship between h and Energy deviation (error): error = {const:.3f}*h^({m:.3f})")

ax.set_title("Max Energy deviation for varying step size")
ax.set_xlabel("Step size, h (s)")
ax.set_ylabel("Max deviation (%)")
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
# --------------------------------------------------------------------------------------------
# Plotting Heat map for different initial values
# --------------------------------------------------------------------------------------------

# Parameters
g = 9.81 #m/s^2

m1 = 2 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 1 # m

T = 10 #s
N = 1000

grid = 100

# --------------------------------------------------------------------------------------------
# 1. Varying initial values for mass 2 - larger range
# Initial data
init_th1 = 0
init_th2 = np.linspace(0, 2*np.pi, grid)
init_om1 = -30
init_om2 = np.linspace(-30, 30, grid)

Edev_array = np.zeros((grid, grid), dtype=float)

for i in range(grid):
  for j in range(grid):
    t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls = evolve(T, N, [m1, m2, l1, l2], [init_th1, init_th2[i], init_om1, init_om2[j]])
    dev, percent = get_MaxEdev(theta1_ls, theta2_ls, omega1_ls, omega2_ls, [m1, m2, l1, l2])
    #-----
    Edev_array[i,j] = percent
    #-----

fig, ax = plt.subplots(figsize=(7, 7))
im = ax.imshow(Edev_array)

# Show all ticks and label them with the respective list entries
ticks = np.arange(0, grid, 5) #ticks for every 5 values
ax.set_yticks(ticks)
ax.set_yticklabels([f"{init_th2[i]:.2f}" for i in ticks])
ax.set_xticks(ticks)
ax.set_xticklabels([f"{init_om2[i]:.2f}" for i in ticks],
                   rotation=45, ha="right", rotation_mode="anchor")

# Create colorbar
cbar = ax.figure.colorbar(im, ax=ax, shrink=0.6)
cbar.ax.set_ylabel("Max deviation (%)", rotation=-90, va="bottom")

ax.set_title("Max Energy deviation for varying initial values for mass 2")
ax.set_ylabel(r"$\theta_2$")
ax.set_xlabel(r"$\omega_2$")
fig.tight_layout()
plt.show()


In [ ]:
# --------------------------------------------------------------------------------------------
# Plotting Heat map for different initial values
# --------------------------------------------------------------------------------------------

# Parameters
g = 9.81 #m/s^2

m1 = 2 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 1 # m

T = 10 #s
N = 1000

grid = 100

# --------------------------------------------------------------------------------------------
# 2. Varying initial angular velocities - larger values
# Initial data
init_th1 = 0
init_th2 = np.pi
init_om1 = np.linspace(-30, 30, grid)
init_om2 = np.linspace(-30, 30, grid)

Edev_array = np.zeros((grid, grid), dtype=float)

for i in range(grid):
  for j in range(grid):
    t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls = evolve(T, N, [m1, m2, l1, l2], [init_th1, init_th2, init_om1[i], init_om2[j]])
    dev, percent = get_MaxEdev(theta1_ls, theta2_ls, omega1_ls, omega2_ls, [m1, m2, l1, l2])
    #-----
    Edev_array[i,j] = percent
    #-----


fig, ax = plt.subplots(figsize=(7, 7))
im = ax.imshow(Edev_array)

# Show all ticks and label them with the respective list entries
ticks = np.arange(0, grid, 5) #ticks for every 5 values
ax.set_yticks(ticks)
ax.set_yticklabels([f"{init_om1[i]:.2f}" for i in ticks])
ax.set_xticks(ticks)
ax.set_xticklabels([f"{init_om2[i]:.2f}" for i in ticks],
                   rotation=45, ha="right", rotation_mode="anchor")


# Create colorbar
cbar = ax.figure.colorbar(im, ax=ax, shrink=0.5)
cbar.ax.set_ylabel("Max deviation (%)", rotation=-90, va="bottom")

ax.set_title("Max Energy deviation for varying initial angular velocities")
ax.set_ylabel(r"$\omega_1$")
ax.set_xlabel(r"$\omega_2$")
fig.tight_layout()
plt.show()

In [ ]:
#------------------------------------------------------------
# Solve Double Pendulum - ex 3 (high error)
#------------------------------------------------------------

# Parameters and initial data ----------------------------------------------
g = 9.81 #m/s^2

m1 = 2 #kg
m2 = 1 #kg
l1 = 2 # m
l2 = 1 # m

T = 10 #s
N = 1000

initial3 = np.array([0, np.pi, 30, -30])

# Solve the pendulum motion and plot energy ---------------------------------

param3 = [m1, m2, l1, l2]
t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls = evolve(T, N, param3, initial3)

plot_energy(t_ls, theta1_ls, theta2_ls, omega1_ls, omega2_ls, initial3)

animation_ext = animate(theta1_ls, theta2_ls)

animation_ext.save(f"ExtremeErrorCase_DoublePendulum_param={param3}_initial={initial3}.mp4", fps=40, writer="ffmpeg")
animation_ext